In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle datasets download -d borismarjanovic/price-volume-data-for-all-us-stocks-etfs

Dataset URL: https://www.kaggle.com/datasets/borismarjanovic/price-volume-data-for-all-us-stocks-etfs
License(s): CC0-1.0
price-volume-data-for-all-us-stocks-etfs.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('price-volume-data-for-all-us-stocks-etfs.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import pandas as pd
tickers = pd.read_csv('SP Tickers.csv')
tickers = set(tickers['symbol'].tolist())

In [ ]:
import os
import pandas as pd
from pandas.errors import EmptyDataError

dataframes = []
directory = os.path.join(os.getcwd() + '/Stocks')

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        ticker = filename.replace('.us.txt', '').upper()
        file_path = os.path.join(directory, filename)

        if ticker not in tickers: # if the ticker isnt in our list, ignore it
          continue

        try:
            df = pd.read_csv(file_path)
            df['Ticker'] = ticker
            df.sort_values('Date', inplace=True)
            dataframes.append(df)
        except EmptyDataError: # some of the files have no contents so read_csv throws error
          pass

stock_df = pd.concat(dataframes, ignore_index=True)

stock_df['Date'] = pd.to_datetime(stock_df['Date'], format='%Y-%m-%d')

In [ ]:
stock_df.tail()

In [ ]:
TRAINING_START_DATE = pd.to_datetime("2005-01-01")
TRAINING_END_DATE = pd.to_datetime("2012-12-31")

In [ ]:
!pip install ta

In [ ]:
import ta

def get_features_df(stock_df, tickers, start_date, end_date):
  dataframes = []
  for ticker in tickers:
      ticker_df = stock_df.loc[(stock_df['Ticker'] == ticker) & (stock_df['Date'] >= start_date) & (stock_df['Date'] <= end_date)].copy()

      if not ticker_df.empty:
          # Generate all TA features
          ticker_df = ta.add_all_ta_features(
              ticker_df, open="Open", high="High", low="Low", close="Close", volume="Volume"
          )
          dataframes.append(ticker_df)

  df = pd.concat(dataframes, ignore_index=True)
  df.drop(['trend_psar_up', 'trend_psar_down'], axis=1, inplace=True) # these 2 generate issues with NaN entries
  df.dropna(inplace=True)
  return df

In [ ]:
stock_df = get_features_df(stock_df, tickers, TRAINING_START_DATE, TRAINING_END_DATE)

In [ ]:
import numpy as np
stock_df['trading_signal'] = np.where(stock_df['Close'] >= stock_df['Open'], 1, -1)

In [ ]:
dataframes = []
for ticker in tickers:
    ticker_df = stock_df.loc[(stock_df['Ticker'] == ticker)]

    if not ticker_df.empty:
        for row in range(1, len(ticker_df)):
            ticker_df.iloc[row-1, -1] = ticker_df.iloc[row, -1]

        ticker_df.drop(ticker_df.tail(1).index,inplace=True) # drop last row
        dataframes.append(ticker_df)

train_df = pd.concat(dataframes, ignore_index=True)

In [ ]:
train_df.head()